In [1]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import CuDNNLSTM
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization as BatchNorm
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from tqdm import tqdm

music21: Certain music21 functions might need these optional packages: matplotlib, scipy;
                   if you run into errors, install them by following the instructions at
                   http://mit.edu/music21/doc/installing/installAdditional.html


In [ ]:
!pip install music21


In [2]:
notes = []

for file in tqdm(glob.glob("midi/*.mid")):
    midi = converter.parse(file)
    notes_to_parse = None

    try: # file has instrument parts
        s2 = instrument.partitionByInstrument(midi)
        notes_to_parse = s2.parts[0].recurse() 
    except: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 37/37 [00:20<00:00,  1.85it/s]


In [4]:
#Save Notes
with open('notes.pkl', 'wb') as f:
    pickle.dump(notes, f)

In [14]:
#Load Notes
with open('notes.pkl', 'rb') as f:
    notes = pickle.load(f) 

In [3]:
n = len(set(notes))


270

In [4]:
sequence_length = 100


pitchnames = sorted(set(item for item in notes))
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

network_input = []
network_output = []


for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

n_patterns = len(network_input)


network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))

network_input = network_input / float(n)

network_output = to_categorical(network_output)

In [22]:
network_input.shape

(104939, 100, 1)

In [18]:
network_output.shape

(104939, 270)

In [8]:
#Create Model
model = Sequential()
model.add(CuDNNLSTM(256, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))              
model.add(CuDNNLSTM(128, return_sequences=True))
model.add(CuDNNLSTM(64))

model.add(Dense(n_vocab))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [12]:
#model.load_weights("weight/weights-improvement-400-0.5798-bigger.hdf5")
filepath = "weight/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(
    filepath,
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)
callbacks_list = [checkpoint]

model.fit(network_input, network_output, epochs=1000, batch_size=128, callbacks=callbacks_list, initial_epoch=400)

Train on 104939 samples
Epoch 401/1000
104939/104939 [==============================] - 23s 216us/sample - loss: 0.5810
Epoch 402/1000
104939/104939 [==============================] - 24s 233us/sample - loss: 0.5793
Epoch 403/1000
104939/104939 [==============================] - 25s 239us/sample - loss: 0.5755 - - ETA: 0s - lo
Epoch 404/1000
104939/104939 [==============================] - 23s 221us/sample - loss: 0.5756
Epoch 405/1000
104939/104939 [==============================] - 24s 225us/sample - loss: 0.5755
Epoch 406/1000
104939/104939 [==============================] - 24s 227us/sample - loss: 0.5739- ETA: 
Epoch 407/1000
104939/104939 [==============================] - 23s 221us/sample - loss: 0.5701 - loss: 0
Epoch 408/1000
104939/104939 [==============================] - 23s 224us/sample - loss: 0.5749 - loss: 0 - ETA: 6s - ETA:
Epoch 409/1000
104939/104939 [==============================] - 23s 223us/sample - loss: 0.5714 - loss: 0.571 - ETA: 0s - loss: 0.5
Epoch 410/1000


104939/104939 [==============================] - 24s 226us/sample - loss: 0.4483
Epoch 566/1000
104939/104939 [==============================] - 24s 225us/sample - loss: 0.4447
Epoch 567/1000
104939/104939 [==============================] - 24s 228us/sample - loss: 0.4411
Epoch 568/1000
104939/104939 [==============================] - 24s 225us/sample - loss: 0.4456
Epoch 569/1000
104939/104939 [==============================] - 23s 222us/sample - loss: 0.4444
Epoch 570/1000
104939/104939 [==============================] - 23s 221us/sample - loss: 0.4420
Epoch 571/1000
104939/104939 [==============================] - 23s 221us/sample - loss: 0.4442
Epoch 572/1000
104939/104939 [==============================] - 23s 222us/sample - loss: 0.4474
Epoch 573/1000
104939/104939 [==============================] - 24s 225us/sample - loss: 0.4424
Epoch 574/1000
104939/104939 [==============================] - 24s 231us/sample - loss: 0.4430
Epoch 575/1000
104939/104939 [=========================

104939/104939 [==============================] - 25s 238us/sample - loss: 0.3706
Epoch 732/1000
104939/104939 [==============================] - 25s 237us/sample - loss: 0.3712
Epoch 733/1000
104939/104939 [==============================] - 25s 238us/sample - loss: 0.3725
Epoch 734/1000
104939/104939 [==============================] - 26s 245us/sample - loss: 0.3697
Epoch 735/1000
104939/104939 [==============================] - 27s 257us/sample - loss: 0.3720
Epoch 736/1000
104939/104939 [==============================] - 24s 230us/sample - loss: 0.3711
Epoch 737/1000
104939/104939 [==============================] - 24s 224us/sample - loss: 0.3664
Epoch 738/1000
104939/104939 [==============================] - 23s 217us/sample - loss: 0.3636
Epoch 739/1000
104939/104939 [==============================] - 23s 224us/sample - loss: 0.3698
Epoch 740/1000
104939/104939 [==============================] - 24s 225us/sample - loss: 0.3703
Epoch 741/1000
104939/104939 [=========================

104939/104939 [==============================] - 24s 226us/sample - loss: 0.3152
Epoch 897/1000
104939/104939 [==============================] - 24s 224us/sample - loss: 0.3171
Epoch 898/1000
104939/104939 [==============================] - 24s 225us/sample - loss: 0.3146
Epoch 899/1000
104939/104939 [==============================] - 24s 225us/sample - loss: 0.3147
Epoch 900/1000
104939/104939 [==============================] - 24s 227us/sample - loss: 0.3170
Epoch 901/1000
104939/104939 [==============================] - 24s 226us/sample - loss: 0.3143
Epoch 902/1000
104939/104939 [==============================] - 23s 222us/sample - loss: 0.3167
Epoch 903/1000
104939/104939 [==============================] - 23s 222us/sample - loss: 0.3144
Epoch 904/1000
104939/104939 [==============================] - 23s 222us/sample - loss: 0.3139
Epoch 905/1000
104939/104939 [==============================] - 24s 230us/sample - loss: 0.3155
Epoch 906/1000
104939/104939 [=========================